In [22]:
import numpy as np
from transformers import AutoTokenizer
import csv

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

# Specify the file path of the .npy file
# file_path = "tmp/test/generations/0.npy"
file_path = "datasets/train_prefix.npy"

# Load the .npy file
data = np.load(file_path)

# View the first 10 entries
print(data[:1])

print(data.shape)

print(tokenizer.convert_ids_to_tokens(data[0]))

# Specify the file path to save the decoded strings
output_file_path = "./tmp/test/decoded_strings.csv"

# Open the CSV file in write mode
with open(output_file_path, "w", newline='') as file:
    writer = csv.writer(file)

    # Iterate over each entry in the data numpy array
    for ids in data:
        # Decode the entry and replace 'Ġ' and 'ĠÄ' with ''
        decoded_text = tokenizer.decode(ids, skip_special_tokens=True).replace('Ġ', '').replace("ĠÄ", '')
        
        # Write the decoded text to a new row in the CSV file
        writer.writerow([decoded_text])

print("Tokenized strings saved to:", output_file_path)

print("done")

[[ 1640 34754   254    64 34754   254 40940 34754   254 12501  1166 34754
    254   261 34754   254  1169 34754   254  1102  4717 34754   254  1659
  34754   254  1169 34754   254 10262   302  3196 34754   254 23395 34754
    254  1169 34754   254 22030 34754   254 20012 34754   254   392 34754
    254   896]]
(282, 50)
['for', 'ĠÄ', 'ł', 'a', 'ĠÄ', 'ł', 'Council', 'ĠÄ', 'ł', 'dec', 'ision', 'ĠÄ', 'ł', 'on', 'ĠÄ', 'ł', 'the', 'ĠÄ', 'ł', 'con', 'clusion', 'ĠÄ', 'ł', 'of', 'ĠÄ', 'ł', 'the', 'ĠÄ', 'ł', 'Ag', 'Ġre', 'ements', 'ĠÄ', 'ł', 'between', 'ĠÄ', 'ł', 'the', 'ĠÄ', 'ł', 'European', 'ĠÄ', 'ł', 'Community', 'ĠÄ', 'ł', 'and', 'ĠÄ', 'ł', 'its']
Tokenized strings saved to: ./tmp/test/decoded_strings.csv
done
